In [0]:
# for parsing rdd rows
# Columns:
# 0: logging level, 1: timestamp, 2: downloader id, 
# 3: retrieval stage, 4: Action?
def myParse(line):
    line = line.replace(' -- ', ', ')
    line = line.replace('.rb: ', ', ')
    line = line.replace(', ghtorrent-', ', ')
    return line.split(', ', 4)

In [0]:
def getRDD(filename):
    textFile = sc.textFile("/FileStore/tables/ghtorrent_logs.txt")
    parsedRDD = textFile.map(myParse)
    return parsedRDD

In [0]:
rowrdd = getRDD("ghtorrent_logs.txt").cache()


In [0]:
print(rowrdd.count())

print("Number of Lines in RDD:" + str(rowrdd.count()))


281234
Number of Lines in RDD:281234


In [0]:
numWarn = rowrdd.filter(lambda x: x[0] == "WARN")
print(numWarn.count())

print("Total Number of WARNing messages in File are :" + str(numWarn.count()))

3811
Total Number of WARNing messages in File are :3811


In [0]:
import itertools

# rewrite with split, and use only api_client
def parseRepos(x):
    try:
        split = x[4].split('/')[4:6]
        joinedSplit = '/'.join(split)
        result = joinedSplit.split('?')[0]
    except: 
        result = ''
    x.append(result)
    return x

# Filters out rows without enough elements (about 50 rows)
filteredRdd = rowrdd.filter(lambda x: len(x) == 5) 

# Only look at api_client calls
apiRdd = filteredRdd.filter(lambda x: x[3] == "api_client")

# Add another column with the repo if can find one, otherwise ''
reposRdd = apiRdd.map(parseRepos)


In [0]:
# Filter out rows without repo
removedEmpty = reposRdd.filter(lambda x: x[5] != '')
# Group by repo and count
uniqueRepos = removedEmpty.groupBy(lambda x: x[5])
print(uniqueRepos.count())


16532


In [0]:
# filter failed http requests
onlyFailed = apiRdd.filter(lambda x: x[4].split(' ', 1)[0] == "Failed")
# Group by, count, find max
usersFailedHttp = onlyFailed.groupBy(lambda x: x[2])
usersFailedHttpSum = usersFailedHttp.map(lambda x: (x[0], x[1].__len__()))
print(usersFailedHttpSum.max(key=lambda x: x[1]))


('13', 2321)


In [0]:
# Get hour of the day from timestamp and add it
def appendAndReturn(x, toAdd):
    x.append(toAdd)
    return x

# Split date to hour only
onlyHours = filteredRdd.map(lambda x: appendAndReturn(x, x[1].split('T', 1)[1].split(':', 1)[0]))
# Group by, count, find max
groupOnlyHours = onlyHours.groupBy(lambda x: x[5])
hoursCount = groupOnlyHours.map(lambda x: (x[0], x[1].__len__()))
print(hoursCount.max(key=lambda x: x[1]))


('10', 78000)


In [0]:
activityRepos = removedEmpty.groupBy(lambda x: x[5])
countActivityRepos = activityRepos.map(lambda x: (x[0], x[1].__len__()))
print(countActivityRepos.max(key=lambda x: x[1]))



('greatfakeman/Tabchi', 2318)
